# 基于无人机影像的车辆旋转检测

本项目是基于PaddleDetection现有的三个旋转框检测模型：ppyoloe_r、fcosr和s2anet去实现无人机影像的车辆旋转检测。

我们可以看到，单从基于无人机影像的车辆旋转检测这个任务上来看，PaddleDetection现有的三个模型性能是差不多的，都能够达到90%以上的mAP,能够满足实际检测需要的精度。
| 模型名称 | mAP | FPS |
| -------- | -------- | -------- |
| ppyoloe_r     | 90.89%     | 8.30    |
| fcosr     | 90.90%     | 8.04     |
| s2anet     | 90.73%     | 7.34    |

## 一、项目背景

- 无人机具有**成本低、灵活性高、操作简单、体积小**等优点，可以弥补卫星和载人航空遥感技术的不足，催生了更加多元化的应用场景，其中一个就是使用无人机对复杂路口处进行观测。无人机影像的智能化分析处理不仅可以快速高效地提取地物信息，还能拓展无人机的场景理解能力。
- 无人机航拍由于成像视角不同于自然场景图像，一般有以下特点：
    * **背景复杂**。无人机的拍摄视角和更大的幅宽可以获取到更丰富的地物信息，但这种无法突显目标的拍摄方式也给检测任务带来了噪声干扰。同时，由于无人机的飞行高度相对较低，空域环境较为复杂，因此遮挡现象在无人机航拍影像中较为常见，导致无人机对目标的观测往往具有不连续性和不完整性。
    * **小目标**。无人机图像中的目标尺度变化大，且小目标的比例远高于自然场景图像。
    * **大视场**。大幅宽下的影像往往包含着稀疏不均的目标分布，搜索目标需要花费更高的成本。
    * **旋转**。目标的朝向是任意的，同一类别目标的朝向角度也不相同。
- 本项目使用**UAV-ROD数据集**，基于PaddleDetection提供的旋转目标检测模型对路口影像中的车辆进行检测。

## 二、数据集介绍与处理

- 本项目使用数据集为UAV-ROD，是国科大提供的面向无人机的汽车数据集，带有定向边界框，AI Studio数据集链接：[UAV-ROD](https://aistudio.baidu.com/aistudio/datasetdetail/129957)

- **数据简介**：
    - UAV-ROD由1577张图像和30090个汽车类别实例组成，这些实例由定向边界框标注。训练集和测试集中的图像数分别为1150和427。
    - 国科大使用无人机收集图像，无人机的飞行高度在30米到80米之间。
    - 图像场景包括城市道路、停车场、居民区、路边等。
    - **注**：在UAV-ROD中，θ表示车头方向的角度，并按顺时针方向增加。w和h分别代表定向框的宽度和高度。
    - 在UAV-ROD数据集中，每张图像的平均对象数为19.08，其中每张图像的最大对象数为104。

![](https://ai-studio-static-online.cdn.bcebos.com/35988c744b7e4c359e23b03ce4ab7e967f0d5b01f70743c0a972a2cc81a046d8)

## 三、数据预处理

**Step01：** 解压数据集

In [ ]:
!unzip /home/aistudio/data/data129957/UAV_ROD_Data.zip -d /home/aistudio/work/

**Step02：** 图像尺寸分析

通过图像尺寸分析，我们可以看到该数据集图片共有两种尺寸，分别是：
* [2720, 1530]
* [1920, 1080]

在旋转框检测中，模型输入的尺寸是[1024, 1024]，该数据集的图片并不是非常的大，因此我认为没有必要去离线切图然后再训练。

In [ ]:
import os
from unicodedata import name
import xml.etree.ElementTree as ET
import glob

def Image_size(indir):
    # 提取xml文件列表
    os.chdir(indir)
    annotations = os.listdir('.')
    annotations = glob.glob(str(annotations) + '*.xml')
    width_heights = []

    for i, file in enumerate(annotations): # 遍历xml文件
        # actual parsing
        in_file = open(file, encoding = 'utf-8')
        tree = ET.parse(in_file)
        root = tree.getroot()
        width = int(root.find('size').find('width').text)
        height = int(root.find('size').find('height').text)
        if [width, height] not in width_heights: width_heights.append([width, height])
    print("数据集中，有{}种不同的尺寸，分别是：".format(len(width_heights)))
    for item in width_heights:
        print(item)

indir='/home/aistudio/work/UAV_ROD_Data/train/annotations/'   # xml文件所在的目录
Image_size(indir)

**Step3：** 数据集格式

旋转框使用标准COCO数据格式，你可以将你的数据集转换成COCO格式以训练模型。COCO标准数据格式的标注信息中包含以下信息：
```
'annotations': [
    {
        'id': 2083, 'category_id': 9, 'image_id': 9008,
        'bbox': [x, y, w, h], # 水平框标注
        'segmentation': [[x1, y1, x2, y2, x3, y3, x4, y4]], # 旋转框标注
        ...
    }
    ...
]
```

需要注意的是bbox的标注是水平框标注，segmentation为旋转框四个点的标注(顺时针或逆时针均可)。在旋转框训练时bbox是可以缺省，一般推荐根据旋转框标注segmentation生成。 在PaddleDetection 2.4及之前的版本，bbox为旋转框标注[x, y, w, h, angle]，segmentation缺省，目前该格式已不再支持，请下载最新数据集或者转换成标准COCO格式。

在本任务中，xml格式的标注是[x, y, w, h, angle]的旋转框标注格式，这不能使用现有的PaddleDetection套件直接训练，因此需要根据某种方法去转换。而json格式的标注已经有segmentation字段旋转框标注，所以我们可以直接使用COCO格式数据集训练，在接下来的操作中我使用的也是COCO格式的数据集。

## 四、环境配置

### 4.1 安装PaddleDetection

In [ ]:
# 克隆PaddleDetection仓库
!git clone https://github.com/PaddlePaddle/PaddleDetection.git

# 安装其他依赖
%cd /home/aistudio/PaddleDetection/
!pip install -r requirements.txt --user

# 编译安装paddledet
!python setup.py install

### 4.2 安装依赖

旋转框检测模型需要依赖外部算子进行训练，评估等。Linux环境下，你可以执行以下命令进行编译安装：

In [ ]:
%cd ppdet/ext_op
!python setup.py install

判断是否安装成功。

In [ ]:
%cd unittest 
!python test_matched_rbox_iou.py

In [ ]:
!python test_rbox_iou.py

## 五、代码实现

### 5.1 模型介绍

PP-YOLOE-R 以 CSPRepResNet 作为 backbone，以 CSPPAN 作为 neck，引出 P3、P4 和 P5 三个特征图做检测，同时引入了一个解耦的角度预测头。
1. Baseline：借鉴了 FCOSR 的做法，将 Assigner 和 ProbIoU 损失函数引入到了 PP-YOLOE-l 中作为 baseline。backbone 和 neck 均保持不变，回归分支被修改为直接预测（x,y,w,h,θ）。
2. Rotated Task Alignment Learning：TAL（Task Alignment Learning）是在 PP-YOLOE 中使用的动态匹配算法，为了将其应用到旋转框中，PaddleDetection团队对 TAL 做出了两个简单的适配，得到了 Rotated TAL。首先，利用点与旋转矩形框的几何性质判断点是否在矩形框内部，来筛选 Anchor Points。其次，计算了预测的旋转矩形框与 GT 框的旋转 IoU，加入 metric 的计算中。
3. 解耦的角度预测头：目前大多数的旋转框检测模型直接在一个回归分支中预测（x,y,w,h,θ）。然而，PaddleDetection团队认为（x,y,w,h）和 θ 需要不同的特征，因此将其进行解耦，设计了一个独立的轻量级角度预测分支，仅包含一个 ESE（Effective Squeeze and Extraction）注意力模块和一层卷积层。
4. 使用 DFL（Distribution Focal Loss）进行角度预测：ProbIoU 等损失函数将旋转矩形框建模成高斯矩形框，然后利用两个高斯矩形框之间的距离等度量作为回归损失。然而，对于接近方形的旋转框，将其转变成高斯矩形框后，角度信息将丢失，导致对于接近方形的旋转框的角度预测不准。为了解决这一问题，使用了 DFL 的方式预测角度。不同于 L1 或者 L2 损失函数，DFL 直接学习角度的通用分布。将[0,π/2]的角度区间分成90份，每一份的区间大小为 π/180。通过 DFL 学习角度落在每一个区间的概率，然后利用积分得到预测的角度值。
5. 可学习的门控单元：RepVGG 提出了一个多分支的卷积结构，包含一个3x3的卷积和1x1的卷积以及一个残差连接。在推理阶段，RepVGG 可以被重参数化为一个3x3卷积。虽然 RepVGG 等价于一个卷积，但是在训练时使用 RepVGG 模型收敛的更好。PaddleDetection团队将这一结果归功于 RepVGG 引入了有用的先验知识。受此启发，设计了一个可学习的门控单元来控制来自不同层的信息。由于我们提出的 RepResBlock 中不使用残差连接，因此只需为每一个 RepResBlock 引入一个可学习的参数。在推理时，可以将其与卷积层一起重参数化，保持推理时的参数量和计算量都不变。
![](https://ai-studio-static-online.cdn.bcebos.com/369dffe0b55049ebb44afa4b46197d277684171f4c864598bf8230ece990329f)

### 5.2 模型训练

**Step01：** 将数据集UAV_ROD_Data移动到/home/aistudio/PaddleDetection/dataset目录下。

In [5]:
!mv /home/aistudio/work/* /home/aistudio/PaddleDetection/dataset/

**Step02：** 单卡训练

工作目录：/home/aistudio/PaddleDetection/dataset/UAV_ROD_Data。

将uav_rod_train.json移动到train目录下。

将uav_rod_test.json移动到test目录下。

**ppyoloe_r**

In [ ]:
%cd /home/aistudio/PaddleDetection/
!python tools/train.py -c configs/rotate/ppyoloe_r/ppyoloe_r_crn_l_3x_dota.yml --eval --use_vdl True --vdl_log_dir vdl_log_dir/scalar

损失函数如图所示：

![](https://ai-studio-static-online.cdn.bcebos.com/7c1e94df59bf4271aaa2bb709aff852bfce406a5207e4992b7bb164c28845899)

![](https://ai-studio-static-online.cdn.bcebos.com/bffb78fce6594bdf8f01c4d855e24e7c094cdd0b1abf4ae8aee65f984b42be81)

**fcosr**

FCOSR是基于FCOS的单阶段Anchor-Free的旋转框检测算法。FCOSR主要聚焦于旋转框的标签匹配策略，提出了椭圆中心采样和模糊样本标签匹配的方法。在loss方面，FCOSR使用了ProbIoU避免边界不连续性问题。

In [ ]:
%cd /home/aistudio/PaddleDetection/
!python tools/train.py -c configs/rotate/fcosr/fcosr_x50_3x_dota.yml --eval --use_vdl True --vdl_log_dir vdl_log_dir/scalar

损失函数如图所示：

![](https://ai-studio-static-online.cdn.bcebos.com/bdc09b18176c4f1fb18e7426ff043d0e30eb8a29434741b0af58a3c960b41a32)

**s2anet**

S2ANet是用于检测旋转框的模型.

In [ ]:
%cd /home/aistudio/PaddleDetection/
!python tools/train.py -c configs/rotate/s2anet/s2anet_alignconv_2x_dota.yml --eval --use_vdl True --vdl_log_dir vdl_log_dir/scalar

损失函数如图所示：

![](https://ai-studio-static-online.cdn.bcebos.com/d67e6ee8aa054825a90c6146ccdaeb2216a0edc983134f09929a08be2c078ac5)

### 5.3 模型评估

通过如下命令在单个GPU上评估我们的验证集。

**ppyoloe_r：**

* mAP(0.50, 11point) = 90.89%
* average FPS: 8.295098923380092

In [ ]:
%cd /home/aistudio/PaddleDetection/
!python tools/eval.py -c configs/rotate/ppyoloe_r/ppyoloe_r_crn_l_3x_dota.yml -o weights=output/ppyoloe_r_crn_l_3x_dota/best_model.pdparams

**fcosr：**

* mAP(0.50, 11point) = 90.90%
* average FPS: 8.043081151665614

In [ ]:
%cd /home/aistudio/PaddleDetection/
!python tools/eval.py -c configs/rotate/fcosr/fcosr_x50_3x_dota.yml -o weights=output/fcosr_x50_3x_dota/best_model.pdparams

**s2anet：**

* mAP(0.50, 11point) = 90.73%
* average FPS: 7.342212998352953

In [ ]:
%cd /home/aistudio/PaddleDetection/
!python tools/eval.py -c configs/rotate/s2anet/s2anet_alignconv_2x_dota.yml -o weights=output/s2anet_alignconv_2x_dota/best_model.pdparams

我们可以看到，在基于无人机影像的车辆旋转检测任务中，PaddleDetection现有的三个模型性能是差不多的。
| 模型名称 | mAP | FPS |
| -------- | -------- | -------- |
| ppyoloe_r     | 90.89%     | 8.295098923380092     |
| fcosr     | 90.90%     | 8.043081151665614     |
| s2anet     | 90.73%     | 7.342212998352953     |



### 5.4 模型推理

以ppyoloe_r为例，我们可以通过以下命令在单张GPU上推理文件中的所有图片。

In [ ]:
!python tools/infer.py -c configs/rotate/ppyoloe_r/ppyoloe_r_crn_l_3x_dota.yml -o weights=output/ppyoloe_r_crn_l_3x_dota/best_model.pdparams --infer_dir=dataset/UAV_ROD_Data/test/images --output_dir infer_output/

将部分图片推理结果合并展示，代码如下所示：

In [4]:
import os
from PIL import Image
 
IMAGES_PATH = '/home/aistudio/PaddleDetection/infer_output/'  # 图片集地址
IMAGES_FORMAT = ['.png']  # 图片格式
IMAGE_SIZE = [960, 540]  # 每张小图片的大小
IMAGE_ROW = 2  # 图片间隔，也就是合并成一张图后，一共有几行
IMAGE_COLUMN = 2  # 图片间隔，也就是合并成一张图后，一共有几列
IMAGE_SAVE_PATH = '/home/aistudio/work/result.png'  # 图片转换后的地址
 
# 获取图片集地址下的所有图片名称
image_names = [name for name in os.listdir(IMAGES_PATH) for item in IMAGES_FORMAT if
               os.path.splitext(name)[1] == item]

# 简单的对于参数的设定和实际图片集的大小进行数量判断
if len(image_names) < IMAGE_ROW * IMAGE_COLUMN:
    raise ValueError("合成图片的参数和要求的数量不能匹配！")
 
# 定义图像拼接函数
def image_compose():
    to_image = Image.new('RGBA', (IMAGE_COLUMN * IMAGE_SIZE[0], IMAGE_ROW * IMAGE_SIZE[1])) #创建一个新图
    # 循环遍历，把每张图片按顺序粘贴到对应位置上
    for y in range(1, IMAGE_ROW + 1):
        for x in range(1, IMAGE_COLUMN + 1):
            from_image = Image.open(IMAGES_PATH + image_names[IMAGE_COLUMN * (y - 1) + x - 1]).resize(
                (IMAGE_SIZE[0], IMAGE_SIZE[1]),Image.ANTIALIAS)
            to_image.paste(from_image, ((x - 1) * IMAGE_SIZE[0], (y - 1) * IMAGE_SIZE[1]))
    return to_image.save(IMAGE_SAVE_PATH) # 保存新图
image_compose() #调用函数

部分可视化结果如下：

![](https://ai-studio-static-online.cdn.bcebos.com/212ab019d548497f85e22cb5f567bfc1a0ede62fb99b4754bde9f9f24c91a680)

### 5.5 模型导出

In [ ]:
%cd /home/aistudio/PaddleDetection
# !python tools/export_model.py -c configs/rotate/ppyoloe_r/ppyoloe_r_crn_l_3x_dota.yml -o weights=output/ppyoloe_r_crn_l_3x_dota/best_model.pdparams
!python tools/export_model.py -c configs/rotate/fcosr/fcosr_x50_3x_dota.yml -o weights=output/fcosr_x50_3x_dota/best_model.pdparams
!python tools/export_model.py -c configs/rotate/s2anet/s2anet_alignconv_2x_dota.yml -o weights=output/s2anet_alignconv_2x_dota/best_model.pdparams

## 六、总结提高

本项目是一个基于无人机影像的旋转框检测任务，模型训练的速度非常地快，同时从精度上来看PaddleDetection的三个旋转框检测模型都能够达到90%以上的mAP，但仍然有部分不足的地方。如果车辆存在部分遮挡或者出现在画面边缘的部分车辆是很难识别到的，后期可以对这些进行进一步的优化。

作者简介：Submerge. 江苏某大学大三学生 人工智能专业 [主页链接](https://aistudio.baidu.com/aistudio/personalcenter/thirdview/2365489) 欢迎互关！

飞桨导师：高睿 [TowerNet](https://aistudio.baidu.com/aistudio/personalcenter/thirdview/634942) 在此感谢。